# Data Preparation 

In [2]:
import pandas as pd
import numpy as np

## I) Für Heating

In [4]:
file_path_data = "data/02_Data_Heating_merged.csv"

In [5]:
df_data_heating = pd.read_csv(file_path_data)
df_data_heating

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,TStart in °C,TEnd in °C,Tm in °C,"Tm,2 in °C",Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Peak Signal Characterisation_Quasibimodal,Aging Level
0,0-PE-1_1,0-PE-6,1.17,109.60,1st Heating,55.10,118.00,110.3,NaN,NaN,x,NaN,l,NaN,x,NaN,NaN,NaN,Pristine LDPE and PET
1,0-PE-1_1,0-PE-6,1.17,108.10,2nd Heating,50.00,118.38,107.7,NaN,NaN,x,NaN,l,NaN,NaN,NaN,NaN,NaN,Pristine LDPE and PET
2,0-PE-1_2,0-PE-7,1.22,106.00,1st Heating,55.00,119.80,108.5,NaN,NaN,x,NaN,l,NaN,x,NaN,NaN,NaN,Pristine LDPE and PET
3,0-PE-1_2,0-PE-7,1.22,107.90,2nd Heating,50.00,119.30,107.7,NaN,NaN,x,NaN,l,NaN,NaN,NaN,NaN,NaN,Pristine LDPE and PET
4,0-PE-1_3,0-PE-8,1.19,110.20,1st Heating,55.00,118.00,110.1,NaN,NaN,x,NaN,l,NaN,x,NaN,NaN,NaN,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,3-EBT-PET-4_3,eb-PET-64,1.30,36.87,2nd Heating,202.37,251.33,240,NaN,NaN,NaN,x,s,NaN,NaN,NaN,NaN,NaN,Third EBT
583,3-EBT-PET-4_4,eb-PET-65,1.21,39.22,1st Heating,209.10,254.46,239.9,NaN,NaN,NaN,x,s,NaN,NaN,NaN,NaN,NaN,Third EBT
584,3-EBT-PET-4_4,eb-PET-65,1.21,36.30,2nd Heating,198.86,256.10,239.3,NaN,NaN,NaN,x,s,NaN,NaN,NaN,NaN,NaN,Third EBT
585,3-EBT-PET-4_5,eb-PET-66,1.15,39.75,1st Heating,209.14,254.32,239.6,NaN,NaN,NaN,x,s,NaN,NaN,NaN,NaN,NaN,Third EBT


### a) Analyse 

In [7]:
df_data_heating.isnull().sum()

Sample Name                                      0
Name of Original DSC Measurement                 0
Mass in mg                                       0
∆Hm in Jg-1                                      0
Segment                                          0
TStart in °C                                     0
TEnd in °C                                       0
Tm in °C                                         0
Tm,2 in °C                                     432
Peak Signal Characterisation_Bimodal           432
Peak Signal Characterisation_Sharp Monodal     427
Peak Signal Characterisation_Wide Monodal      317
Peak Signal Characterisation_Left Tail           2
Peak Signal Characterisation_Right Tail        587
Peak Signal Characterisation_Left Shoulder     507
Peak Signal Characterisation_Right Shoulder    587
Peak Signal Characterisation                   567
Peak Signal Characterisation_Quasibimodal      585
Aging Level                                      0
dtype: int64

In [8]:
df_data_heating.dtypes

Sample Name                                     object
Name of Original DSC Measurement                object
Mass in mg                                     float64
∆Hm in Jg-1                                    float64
Segment                                         object
TStart in °C                                   float64
TEnd in °C                                     float64
Tm in °C                                        object
Tm,2 in °C                                     float64
Peak Signal Characterisation_Bimodal            object
Peak Signal Characterisation_Sharp Monodal      object
Peak Signal Characterisation_Wide Monodal       object
Peak Signal Characterisation_Left Tail          object
Peak Signal Characterisation_Right Tail        float64
Peak Signal Characterisation_Left Shoulder      object
Peak Signal Characterisation_Right Shoulder    float64
Peak Signal Characterisation                    object
Peak Signal Characterisation_Quasibimodal       object
Aging Leve

In [9]:
peak_signal_columns = [col for col in df_data_heating.columns if "Peak Signal" in col]

for col in peak_signal_columns:
    value_counts = df_data_heating[col].value_counts(dropna=False)  # Häufigkeit von Werten, inklusive NaN
    print(value_counts)
    print("---"*20)

Peak Signal Characterisation_Bimodal
NaN    432
x       95
1<2     40
1>2     15
1=2      5
Name: count, dtype: int64
------------------------------------------------------------
Peak Signal Characterisation_Sharp Monodal
NaN    427
x      160
Name: count, dtype: int64
------------------------------------------------------------
Peak Signal Characterisation_Wide Monodal
NaN    317
x      270
Name: count, dtype: int64
------------------------------------------------------------
Peak Signal Characterisation_Left Tail
s      326
l      214
x       45
NaN      2
Name: count, dtype: int64
------------------------------------------------------------
Peak Signal Characterisation_Right Tail
NaN    587
Name: count, dtype: int64
------------------------------------------------------------
Peak Signal Characterisation_Left Shoulder
NaN    507
x       80
Name: count, dtype: int64
------------------------------------------------------------
Peak Signal Characterisation_Right Shoulder
NaN    587
Nam

In [10]:
# Konvertiere 'Tm in °C' in numerische Werte mit to_numeric
df_data_heating['Tm in °C'] = pd.to_numeric(df_data_heating['Tm in °C'], errors='coerce')

# Remove rows with errors in the conversion for the entire dataset
df_data_heating = df_data_heating.dropna(subset=['Tm in °C'])

## b) Missing Value Treatment

### Naive Vorgehensweise: NaN-Werte mit 0 ersetzen für "Peak Signal Characterisation"-Spalten

In [13]:
# Ersetze NaN-Werte in allen Spalten, die "Peak Signal" im Namen enthalten, mit 0
peak_signal_columns = [col for col in df_data_heating.columns if "Peak Signal" in col]
df_data_heating[peak_signal_columns] = df_data_heating[peak_signal_columns].fillna(0)
df_data_heating

C:\Users\konra\AppData\Local\Temp\ipykernel_8660\2454315781.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_heating[peak_signal_columns] = df_data_heating[peak_signal_columns].fillna(0)


,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,TStart in °C,TEnd in °C,Tm in °C,"Tm,2 in °C",Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Peak Signal Characterisation_Quasibimodal,Aging Level
0,0-PE-1_1,0-PE-6,1.17,109.60,1st Heating,55.10,118.00,110.3,NaN,0,x,0,l,0.0,x,0.0,0,0,Pristine LDPE and PET
1,0-PE-1_1,0-PE-6,1.17,108.10,2nd Heating,50.00,118.38,107.7,NaN,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
2,0-PE-1_2,0-PE-7,1.22,106.00,1st Heating,55.00,119.80,108.5,NaN,0,x,0,l,0.0,x,0.0,0,0,Pristine LDPE and PET
3,0-PE-1_2,0-PE-7,1.22,107.90,2nd Heating,50.00,119.30,107.7,NaN,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
4,0-PE-1_3,0-PE-8,1.19,110.20,1st Heating,55.00,118.00,110.1,NaN,0,x,0,l,0.0,x,0.0,0,0,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,3-EBT-PET-4_3,eb-PET-64,1.30,36.87,2nd Heating,202.37,251.33,240.0,NaN,0,0,x,s,0.0,0,0.0,0,0,Third EBT
583,3-EBT-PET-4_4,eb-PET-65,1.21,39.22,1st Heating,209.10,254.46,239.9,NaN,0,0,x,s,0.0,0,0.0,0,0,Third EBT
584,3-EBT-PET-4_4,eb-PET-65,1.21,36.30,2nd Heating,198.86,256.10,239.3,NaN,0,0,x,s,0.0,0,0.0,0,0,Third EBT
585,3-EBT-PET-4_5,eb-PET-66,1.15,39.75,1st Heating,209.14,254.32,239.6,NaN,0,0,x,s,0.0,0,0.0,0,0,Third EBT


### "Tm,2 in °C"-Spalte entfernen, da raltiv wenig Einträge vorhanden sind!

In [15]:
# Entferne die Spalte "Tm,2 in °C" aus dem DataFrame
if "Tm,2 in °C" in df_data_heating.columns:
    df_data_heating.drop(columns=["Tm,2 in °C"], inplace=True)
    print("Die Spalte 'Tm,2 in °C' wurde erfolgreich entfernt.")
else:
    print("Die Spalte 'Tm,2 in °C' existiert nicht im DataFrame.")
df_data_heating    

Die Spalte 'Tm,2 in °C' wurde erfolgreich entfernt.


C:\Users\konra\AppData\Local\Temp\ipykernel_8660\1503624843.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_heating.drop(columns=["Tm,2 in °C"], inplace=True)


,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,TStart in °C,TEnd in °C,Tm in °C,Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Peak Signal Characterisation_Quasibimodal,Aging Level
0,0-PE-1_1,0-PE-6,1.17,109.60,1st Heating,55.10,118.00,110.3,0,x,0,l,0.0,x,0.0,0,0,Pristine LDPE and PET
1,0-PE-1_1,0-PE-6,1.17,108.10,2nd Heating,50.00,118.38,107.7,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
2,0-PE-1_2,0-PE-7,1.22,106.00,1st Heating,55.00,119.80,108.5,0,x,0,l,0.0,x,0.0,0,0,Pristine LDPE and PET
3,0-PE-1_2,0-PE-7,1.22,107.90,2nd Heating,50.00,119.30,107.7,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
4,0-PE-1_3,0-PE-8,1.19,110.20,1st Heating,55.00,118.00,110.1,0,x,0,l,0.0,x,0.0,0,0,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,3-EBT-PET-4_3,eb-PET-64,1.30,36.87,2nd Heating,202.37,251.33,240.0,0,0,x,s,0.0,0,0.0,0,0,Third EBT
583,3-EBT-PET-4_4,eb-PET-65,1.21,39.22,1st Heating,209.10,254.46,239.9,0,0,x,s,0.0,0,0.0,0,0,Third EBT
584,3-EBT-PET-4_4,eb-PET-65,1.21,36.30,2nd Heating,198.86,256.10,239.3,0,0,x,s,0.0,0,0.0,0,0,Third EBT
585,3-EBT-PET-4_5,eb-PET-66,1.15,39.75,1st Heating,209.14,254.32,239.6,0,0,x,s,0.0,0,0.0,0,0,Third EBT


In [16]:
# Versuche alle Spalten in peak_signal_columns in Integer-Datentyp umzuwandeln, ansonsten unverändert lassen
peak_signal_columns = [col for col in df_data_heating.columns if "Peak Signal" in col]

for col in peak_signal_columns:
    # Prüfen, ob die Spalte vollständig in Integer konvertierbar ist
    if df_data_heating[col].apply(lambda x: isinstance(x, (int, float)) and x == int(x)).all():
        df_data_heating.loc[:, col] = df_data_heating[col].astype(int)
        print(f"Spalte '{col}' erfolgreich in Integer umgewandelt.")
        
        
df_data_heating       

Spalte 'Peak Signal Characterisation_Right Tail' erfolgreich in Integer umgewandelt.
Spalte 'Peak Signal Characterisation_Right Shoulder' erfolgreich in Integer umgewandelt.


,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,TStart in °C,TEnd in °C,Tm in °C,Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Peak Signal Characterisation_Quasibimodal,Aging Level
0,0-PE-1_1,0-PE-6,1.17,109.60,1st Heating,55.10,118.00,110.3,0,x,0,l,0.0,x,0.0,0,0,Pristine LDPE and PET
1,0-PE-1_1,0-PE-6,1.17,108.10,2nd Heating,50.00,118.38,107.7,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
2,0-PE-1_2,0-PE-7,1.22,106.00,1st Heating,55.00,119.80,108.5,0,x,0,l,0.0,x,0.0,0,0,Pristine LDPE and PET
3,0-PE-1_2,0-PE-7,1.22,107.90,2nd Heating,50.00,119.30,107.7,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
4,0-PE-1_3,0-PE-8,1.19,110.20,1st Heating,55.00,118.00,110.1,0,x,0,l,0.0,x,0.0,0,0,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,3-EBT-PET-4_3,eb-PET-64,1.30,36.87,2nd Heating,202.37,251.33,240.0,0,0,x,s,0.0,0,0.0,0,0,Third EBT
583,3-EBT-PET-4_4,eb-PET-65,1.21,39.22,1st Heating,209.10,254.46,239.9,0,0,x,s,0.0,0,0.0,0,0,Third EBT
584,3-EBT-PET-4_4,eb-PET-65,1.21,36.30,2nd Heating,198.86,256.10,239.3,0,0,x,s,0.0,0,0.0,0,0,Third EBT
585,3-EBT-PET-4_5,eb-PET-66,1.15,39.75,1st Heating,209.14,254.32,239.6,0,0,x,s,0.0,0,0.0,0,0,Third EBT


### c) Export

In [18]:
df_1st_heating = df_data_heating[df_data_heating["Segment"] == "1st Heating"]
df_1st_heating.to_csv("data/03_Data_1st_Heating.csv", index=False)
df_1st_heating.tail(5)

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,TStart in °C,TEnd in °C,Tm in °C,Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Peak Signal Characterisation_Quasibimodal,Aging Level
577,3-EBT-PET-4_1,eb-PET-62,1.18,37.17,1st Heating,211.10,253.40,240.3,0,0,x,s,0.0,0,0.0,0,0,Third EBT
579,3-EBT-PET-4_2,eb-PET-63,1.27,38.96,1st Heating,210.10,253.03,239.4,0,0,x,s,0.0,0,0.0,0,0,Third EBT
581,3-EBT-PET-4_3,eb-PET-64,1.30,39.44,1st Heating,207.60,254.50,240.1,0,0,x,s,0.0,0,0.0,0,0,Third EBT
583,3-EBT-PET-4_4,eb-PET-65,1.21,39.22,1st Heating,209.10,254.46,239.9,0,0,x,s,0.0,0,0.0,0,0,Third EBT
585,3-EBT-PET-4_5,eb-PET-66,1.15,39.75,1st Heating,209.14,254.32,239.6,0,0,x,s,0.0,0,0.0,0,0,Third EBT


In [19]:
df_1st_heating = df_data_heating[df_data_heating["Segment"] == "2nd Heating"]
df_1st_heating.to_csv("data/03_Data_2nd_Heating.csv", index=False)
df_1st_heating.head(5)

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hm in Jg-1,Segment,TStart in °C,TEnd in °C,Tm in °C,Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Peak Signal Characterisation_Quasibimodal,Aging Level
1,0-PE-1_1,0-PE-6,1.17,108.1,2nd Heating,50.0,118.38,107.7,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
3,0-PE-1_2,0-PE-7,1.22,107.9,2nd Heating,50.0,119.30,107.7,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
5,0-PE-1_3,0-PE-8,1.19,108.1,2nd Heating,50.0,118.90,107.6,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
7,0-PE-1_4,0-PE-9,1.27,109.4,2nd Heating,50.0,118.20,107.7,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET
9,0-PE-1_5,0-PE-10,1.41,107.1,2nd Heating,50.0,119.00,107.8,0,x,0,l,0.0,0,0.0,0,0,Pristine LDPE and PET


## II) Für Cooling

In [21]:
file_path_data = "data/02_Data_Cooling_merged.csv"

df_data_cooling = pd.read_csv(file_path_data)
df_data_cooling

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hc in Jg-1,Segment,TStart in °C,TEnd in °C,Tc in °C,"Tc,2 in °C",Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Aging Level
0,0-PE-1_1,0-PE-6,1.17,-119.10,1st Cooling,36.0,99.77,93.3,NaN,NaN,x,NaN,l,NaN,x,NaN,NaN,Pristine LDPE and PET
1,0-PE-1_2,0-PE-7,1.22,-120.80,1st Cooling,36.0,99.85,93.4,NaN,NaN,x,NaN,l,NaN,x,NaN,NaN,Pristine LDPE and PET
2,0-PE-1_3,0-PE-8,1.19,-119.50,1st Cooling,36.0,99.29,93.4,NaN,NaN,x,NaN,l,NaN,x,NaN,NaN,Pristine LDPE and PET
3,0-PE-1_4,0-PE-9,1.27,-121.30,1st Cooling,36.0,99.30,93.2,NaN,NaN,x,NaN,l,NaN,x,NaN,NaN,Pristine LDPE and PET
4,0-PE-1_5,0-PE-10,1.41,-119.00,1st Cooling,36.0,98.71,93.4,NaN,NaN,x,NaN,l,NaN,x,NaN,NaN,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,3-EBT-PET-4_1,eb-PET-62,1.18,-41.04,1st Cooling,163.2,208.05,188.5,NaN,NaN,NaN,x,s,NaN,NaN,NaN,NaN,Third EBT
290,3-EBT-PET-4_2,eb-PET-63,1.27,-43.18,1st Cooling,163.2,208.50,189.6,NaN,NaN,NaN,x,s,NaN,NaN,NaN,NaN,Third EBT
291,3-EBT-PET-4_3,eb-PET-64,1.30,-43.28,1st Cooling,162.8,207.63,190.2,NaN,NaN,NaN,x,s,NaN,NaN,NaN,NaN,Third EBT
292,3-EBT-PET-4_4,eb-PET-65,1.21,-42.31,1st Cooling,163.2,208.39,190.4,NaN,NaN,NaN,x,s,NaN,NaN,NaN,NaN,Third EBT


In [22]:
# Analyse
df_data_cooling.isnull().sum()

Sample Name                                      0
Name of Original DSC Measurement                 0
Mass in mg                                       0
∆Hc in Jg-1                                      0
Segment                                          0
TStart in °C                                     0
TEnd in °C                                       0
Tc in °C                                         0
Tc,2 in °C                                     274
Peak Signal Characterisation_Bimodal           274
Peak Signal Characterisation_Sharp Monodal     199
Peak Signal Characterisation_Wide Monodal      115
Peak Signal Characterisation_Left Tail          20
Peak Signal Characterisation_Right Tail        294
Peak Signal Characterisation_Left Shoulder     191
Peak Signal Characterisation_Right Shoulder    273
Peak Signal Characterisation                   294
Aging Level                                      0
dtype: int64

In [23]:
# Konvertiere 'Tm in °C' in numerische Werte mit to_numeric
df_data_cooling['Tc in °C'] = pd.to_numeric(df_data_cooling['Tc in °C'], errors='coerce')

# Remove rows with errors in the conversion for the entire dataset
df_data_cooling = df_data_cooling.dropna(subset=['Tc in °C'])


In [24]:
# Ersetze NaN-Werte in allen Spalten, die "Peak Signal" im Namen enthalten, mit 0
peak_signal_columns = [col for col in df_data_cooling.columns if "Peak Signal" in col]
df_data_cooling[peak_signal_columns] = df_data_cooling[peak_signal_columns].fillna(0)
df_data_cooling

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hc in Jg-1,Segment,TStart in °C,TEnd in °C,Tc in °C,"Tc,2 in °C",Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Aging Level
0,0-PE-1_1,0-PE-6,1.17,-119.10,1st Cooling,36.0,99.77,93.3,NaN,0,x,0,l,0.0,x,0,0.0,Pristine LDPE and PET
1,0-PE-1_2,0-PE-7,1.22,-120.80,1st Cooling,36.0,99.85,93.4,NaN,0,x,0,l,0.0,x,0,0.0,Pristine LDPE and PET
2,0-PE-1_3,0-PE-8,1.19,-119.50,1st Cooling,36.0,99.29,93.4,NaN,0,x,0,l,0.0,x,0,0.0,Pristine LDPE and PET
3,0-PE-1_4,0-PE-9,1.27,-121.30,1st Cooling,36.0,99.30,93.2,NaN,0,x,0,l,0.0,x,0,0.0,Pristine LDPE and PET
4,0-PE-1_5,0-PE-10,1.41,-119.00,1st Cooling,36.0,98.71,93.4,NaN,0,x,0,l,0.0,x,0,0.0,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,3-EBT-PET-4_1,eb-PET-62,1.18,-41.04,1st Cooling,163.2,208.05,188.5,NaN,0,0,x,s,0.0,0,0,0.0,Third EBT
290,3-EBT-PET-4_2,eb-PET-63,1.27,-43.18,1st Cooling,163.2,208.50,189.6,NaN,0,0,x,s,0.0,0,0,0.0,Third EBT
291,3-EBT-PET-4_3,eb-PET-64,1.30,-43.28,1st Cooling,162.8,207.63,190.2,NaN,0,0,x,s,0.0,0,0,0.0,Third EBT
292,3-EBT-PET-4_4,eb-PET-65,1.21,-42.31,1st Cooling,163.2,208.39,190.4,NaN,0,0,x,s,0.0,0,0,0.0,Third EBT


In [25]:
# Entferne die Spalte "Tc,2 in °C" aus dem DataFrame
if "Tc,2 in °C" in df_data_cooling.columns:
    df_data_cooling.drop(columns=["Tc,2 in °C"], inplace=True)
    print("Die Spalte 'Tc,2 in °C' wurde erfolgreich entfernt.")
else:
    print("Die Spalte 'Tc,2 in °C' existiert nicht im DataFrame.")
df_data_cooling

Die Spalte 'Tc,2 in °C' wurde erfolgreich entfernt.


,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hc in Jg-1,Segment,TStart in °C,TEnd in °C,Tc in °C,Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Aging Level
0,0-PE-1_1,0-PE-6,1.17,-119.10,1st Cooling,36.0,99.77,93.3,0,x,0,l,0.0,x,0,0.0,Pristine LDPE and PET
1,0-PE-1_2,0-PE-7,1.22,-120.80,1st Cooling,36.0,99.85,93.4,0,x,0,l,0.0,x,0,0.0,Pristine LDPE and PET
2,0-PE-1_3,0-PE-8,1.19,-119.50,1st Cooling,36.0,99.29,93.4,0,x,0,l,0.0,x,0,0.0,Pristine LDPE and PET
3,0-PE-1_4,0-PE-9,1.27,-121.30,1st Cooling,36.0,99.30,93.2,0,x,0,l,0.0,x,0,0.0,Pristine LDPE and PET
4,0-PE-1_5,0-PE-10,1.41,-119.00,1st Cooling,36.0,98.71,93.4,0,x,0,l,0.0,x,0,0.0,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,3-EBT-PET-4_1,eb-PET-62,1.18,-41.04,1st Cooling,163.2,208.05,188.5,0,0,x,s,0.0,0,0,0.0,Third EBT
290,3-EBT-PET-4_2,eb-PET-63,1.27,-43.18,1st Cooling,163.2,208.50,189.6,0,0,x,s,0.0,0,0,0.0,Third EBT
291,3-EBT-PET-4_3,eb-PET-64,1.30,-43.28,1st Cooling,162.8,207.63,190.2,0,0,x,s,0.0,0,0,0.0,Third EBT
292,3-EBT-PET-4_4,eb-PET-65,1.21,-42.31,1st Cooling,163.2,208.39,190.4,0,0,x,s,0.0,0,0,0.0,Third EBT


In [26]:
# Versuche alle Spalten in peak_signal_columns in Integer-Datentyp umzuwandeln, ansonsten unverändert lassen
peak_signal_columns = [col for col in df_data_cooling.columns if "Peak Signal" in col]

for col in peak_signal_columns:
    # Prüfen, ob die Spalte vollständig in Integer konvertierbar ist
    if df_data_cooling[col].apply(lambda x: isinstance(x, (int, float)) and x == int(x)).all():
        df_data_cooling[col] = df_data_cooling[col].astype(int)
        print(f"Spalte '{col}' erfolgreich in Integer umgewandelt.")

        
df_data_cooling

Spalte 'Peak Signal Characterisation_Right Tail' erfolgreich in Integer umgewandelt.
Spalte 'Peak Signal Characterisation' erfolgreich in Integer umgewandelt.


,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hc in Jg-1,Segment,TStart in °C,TEnd in °C,Tc in °C,Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Aging Level
0,0-PE-1_1,0-PE-6,1.17,-119.10,1st Cooling,36.0,99.77,93.3,0,x,0,l,0,x,0,0,Pristine LDPE and PET
1,0-PE-1_2,0-PE-7,1.22,-120.80,1st Cooling,36.0,99.85,93.4,0,x,0,l,0,x,0,0,Pristine LDPE and PET
2,0-PE-1_3,0-PE-8,1.19,-119.50,1st Cooling,36.0,99.29,93.4,0,x,0,l,0,x,0,0,Pristine LDPE and PET
3,0-PE-1_4,0-PE-9,1.27,-121.30,1st Cooling,36.0,99.30,93.2,0,x,0,l,0,x,0,0,Pristine LDPE and PET
4,0-PE-1_5,0-PE-10,1.41,-119.00,1st Cooling,36.0,98.71,93.4,0,x,0,l,0,x,0,0,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,3-EBT-PET-4_1,eb-PET-62,1.18,-41.04,1st Cooling,163.2,208.05,188.5,0,0,x,s,0,0,0,0,Third EBT
290,3-EBT-PET-4_2,eb-PET-63,1.27,-43.18,1st Cooling,163.2,208.50,189.6,0,0,x,s,0,0,0,0,Third EBT
291,3-EBT-PET-4_3,eb-PET-64,1.30,-43.28,1st Cooling,162.8,207.63,190.2,0,0,x,s,0,0,0,0,Third EBT
292,3-EBT-PET-4_4,eb-PET-65,1.21,-42.31,1st Cooling,163.2,208.39,190.4,0,0,x,s,0,0,0,0,Third EBT


In [27]:
df_data_cooling.to_csv("data/03_Data_Cooling.csv", index = False)
df_data_cooling

,Sample Name,Name of Original DSC Measurement,Mass in mg,∆Hc in Jg-1,Segment,TStart in °C,TEnd in °C,Tc in °C,Peak Signal Characterisation_Bimodal,Peak Signal Characterisation_Sharp Monodal,Peak Signal Characterisation_Wide Monodal,Peak Signal Characterisation_Left Tail,Peak Signal Characterisation_Right Tail,Peak Signal Characterisation_Left Shoulder,Peak Signal Characterisation_Right Shoulder,Peak Signal Characterisation,Aging Level
0,0-PE-1_1,0-PE-6,1.17,-119.10,1st Cooling,36.0,99.77,93.3,0,x,0,l,0,x,0,0,Pristine LDPE and PET
1,0-PE-1_2,0-PE-7,1.22,-120.80,1st Cooling,36.0,99.85,93.4,0,x,0,l,0,x,0,0,Pristine LDPE and PET
2,0-PE-1_3,0-PE-8,1.19,-119.50,1st Cooling,36.0,99.29,93.4,0,x,0,l,0,x,0,0,Pristine LDPE and PET
3,0-PE-1_4,0-PE-9,1.27,-121.30,1st Cooling,36.0,99.30,93.2,0,x,0,l,0,x,0,0,Pristine LDPE and PET
4,0-PE-1_5,0-PE-10,1.41,-119.00,1st Cooling,36.0,98.71,93.4,0,x,0,l,0,x,0,0,Pristine LDPE and PET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,3-EBT-PET-4_1,eb-PET-62,1.18,-41.04,1st Cooling,163.2,208.05,188.5,0,0,x,s,0,0,0,0,Third EBT
290,3-EBT-PET-4_2,eb-PET-63,1.27,-43.18,1st Cooling,163.2,208.50,189.6,0,0,x,s,0,0,0,0,Third EBT
291,3-EBT-PET-4_3,eb-PET-64,1.30,-43.28,1st Cooling,162.8,207.63,190.2,0,0,x,s,0,0,0,0,Third EBT
292,3-EBT-PET-4_4,eb-PET-65,1.21,-42.31,1st Cooling,163.2,208.39,190.4,0,0,x,s,0,0,0,0,Third EBT
